In [ ]:
!python -m pip install pyproj # biblioteca para mudança de sistemas de coordenadas

In [3]:
import matplotlib.pyplot as plt

from datetime import datetime
import json

import numpy as np

from pyproj import Proj, transform

In [17]:
# Vamos carregar os pontos (do JSON filtrado) na variável pontos.
arquivo_pontos = "cleaned_trajetoBrasil.json"

with open(arquivo_pontos, "r") as f:
    pontos = f.read()
    pontos = json.loads(pontos)
    pontos = pontos['photos']

In [18]:
# Aqui escolhemos quais pontos iremos usar no intervalo entre [0-321], no caso do trajeto no Brasil

faixa_de_pontos = range(1000,1100)

In [ ]:
# Função que calcula em metros a distância entre dois pontos p1 e p2, com latitudes lat_p1 e lat_p2 
# e longitudes long_p1 e long_p2, respectivamente. 

def haversine(points_object, index_p1, index_p2): 
    r = 6771000
    lat_p1 = points_object[index_p1]['lat']
    lat_p2 = points_object[index_p2]['lat']
    long_p1 = points_object[index_p1]['long']
    long_p2 = points_object[index_p2]['long']
    
    d = 2*r*math.asin(math.sqrt((math.sin((lat_p2 - lat_p1)/2))**2 + math.cos(lat_p1)*math.cos(lat_p2)*(math.sin( (long_p2 - long_p1)/2 ))**2)
    return d


In [ ]:
# Função que calcula em metros a distância entre dois pontos p1 e p2, com latitudes lat_p1 e lat_p2
# e longitudes long_p1 e long_p2, respectivamente. 

def trogonometriaEsferica(points_object, index_p1, index_p2):
    r = 6771000
    lat_p1 = points_object[index_p1]['lat']
    lat_p2 = points_object[index_p2]['lat']
    long_p1 = points_object[index_p1]['long']
    long_p2 = points_object[index_p2]['long']
    
    C = math.sin(lat_p1)*math.sin(lat_p2) + math.cos(long_p2 - long_p1)*math.cos(lat_p1)*math.cos(lat_p2)
    d = r*math.pi*math.acos(C)
    return d

In [ ]:
# Cálculo da distância entre dois pontos p1 e p2, com latitudes lat_p1 e lat_p2
# e longitudes long_p1 e long_p2, respectivamente. 

def get_point_coords_proj(index, points_object):
    """
    Essa função recebe um índice numérico correspondendo 
    a uma posição na lista de pontos "points_object".
    
    Usa os pontos na projeção EPSG:3857 em
    que a unidade de medida é em metros e portanto
    podemos calcular a distância euclidiana entre dois
    pontos com base em suas coordenadas.
    
    Os pontos retornados são um vetor numpy em que
    a primeira posição é uma medida em metros no eixo
    horizontal e a segunda é num eixo vertical.
    """
    
    lat = points_object[index]['lat']
    lat = float(lat)
    lng = points_object[index]['lng']
    lng = float(lng)
    p = np.array((lng, lat))
    p = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), p[0], p[1])
    return p

def distancia_euclidiana(v1, v2):
    """
    Tendo como parâmetros os vetores numpy 2D 'v1' e 'v2', crie uma
    função que retorne a distância euclidiana entre os dois vetores.
    """
    distancia = ((v1[0] - v2[0])**2 + (v1[1] - v2[1])**2)**(1/2)
    return distancia

